In [3]:
import numpy as np
import pandas as pd

#!pip install pyEDFlib
import pyedflib
#!pip install ipympl

from scipy.fftpack import fft, ifft, fftfreq
from scipy import signal as sg
from scipy.ndimage.filters import gaussian_filter1d, gaussian_filter
from scipy.stats import binned_statistic, entropy, norm
from statsmodels.stats.multitest import multipletests
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

import sys
import os
import time
#import pickle
import dill as pickle

import concurrent.futures

from tqdm.notebook import tqdm
from collections import defaultdict
import itertools

import matplotlib.pyplot as plt
import seaborn as sns
#from IPython.display import display
sns.set(context='notebook', style='darkgrid', palette='deep', font='sans-serif', font_scale=1, color_codes=False, rc=None)
plt.rcParams['figure.figsize'] = (14, 8)

import sys
#print("SYS.PATH: ", sys.path[:3])
#sys.path.insert(0, r"C:\Users\User\[[Python]]\[AlexeyT]\PAC_PROJECT")

sns.set(context='notebook', style='darkgrid', palette='deep', font='sans-serif', font_scale=1, color_codes=False, rc=None)
plt.rcParams['figure.figsize'] = (14, 8)
from utility_functions import *
from lfp_class import LFP
from pac_class import MyPAC
from patient_class import Patient

print("Succesfully imported libraries and modules\n")

Succesfully imported libraries and modules



In [4]:
p2_root_dir = r"C:\Users\aleks\[[Python]]\LAB\Patient2"
#root_dir = r"H:\Alexey_Timchenko\PAC\Patient2"

In [5]:
p2 = Patient(name='Patient2', root_dir=p2_root_dir)
files = p2.find_bdf_files()
for filename in files:
    p2.scan_file_annotations(filename, update_file_conditions=True)

List of things to make sure before analysis: 
1) .bdf files are in patient folder (root_dir)
2) annotation files share the same name as .bdf files but with _annotations.txt suffix
3) annotations share the same naming principle: e.g. 1Day OFF RH (Com)
Looking for .bdf files in C:\Users\aleks\[[Python]]\LAB\Patient2
Found ['Kamalova_STN_postoper_HP_0.5Hz_1day.bdf', 'Kamalova_STN_postoper_HP_0.5Hz_5day.bdf']
Reading  Kamalova_STN_postoper_HP_0.5Hz_1day_annotations.txt
     Onset  Duration             Annotation   Day L-DOPA             State
0     90.0     180.0          1Day OFF Rest  1Day    OFF            [Rest]
1    273.0      14.0    1Day OFF LH (Com) 1  1Day    OFF    [LH, (Com), 1]
2    287.0       9.0  1Day OFF LH (NoCom) 1  1Day    OFF  [LH, (NoCom), 1]
3    356.0      14.0    1Day OFF RH (Com) 1  1Day    OFF    [RH, (Com), 1]
4    371.0      10.0  1Day OFF RH (NoCom) 1  1Day    OFF  [RH, (NoCom), 1]
5    391.0      16.0    1Day OFF LH (Com) 2  1Day    OFF    [LH, (Com), 2]
6    

In [6]:
p2.get_preprocessed_lfps(verbose=False)

Started reading C:\Users\aleks\[[Python]]\LAB\Patient2\Kamalova_STN_postoper_HP_0.5Hz_5day.bdf
Channels: [9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]


Reading Channels: 100%|████████████████████████████████████████████████████████████████| 16/16 [01:28<00:00,  5.55s/it]


EEG A2-R_1-9[A2]
EEG F8-R_2C-15[F
EEG T4-R_2B-12[T
EEG T6-R_2A-10[T
EEG F4-R_3C-16[F
EEG C4-R_3B-14[C
EEG P4-R_3A-11[P
EEG O2-R_4-13[O2
EEG A1-R_1-1[A1]
EEG F7-R_2C-7[F7
EEG T3-R_2B-4[T3
EEG T5-R_2A-2[T5
EEG F3-R_3C-8[F3
EEG C3-R_3B-6[C3
EEG P3-R_3A-3[P3
EEG O1-R_4-5[O1]
Sampling frequency:  16000.0
Downsampling by the factor  8
New sampling frequency:  2000.0
Reading done, 131.0 sec
Started creating bipolar signals

R1-2C created
R1-2B created
R1-2A created
R2C-2B created
R2B-2A created
R2A-2C created
R2C-3C created
R2B-3B created
R2A-3A created
R3C-3B created
R3B-3A created
R3A-3C created
R4-3C created
R4-3B created
R4-3A created
L1-2C created
L1-2B created
L1-2A created
L2C-2B created
L2B-2A created
L2A-2C created
L2C-3C created
L2B-3B created
L2A-3A created
L3C-3B created
L3B-3A created
L3A-3C created
L4-3C created
L4-3B created
L4-3A created


  0%|          | 0/30 [00:00<?, ?it/s]

Started reading C:\Users\aleks\[[Python]]\LAB\Patient2\Kamalova_STN_postoper_HP_0.5Hz_1day.bdf
Channels: [9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]


Reading Channels: 100%|████████████████████████████████████████████████████████████████| 16/16 [01:29<00:00,  5.57s/it]


EEG A2-R_1-9[A2]
EEG F8-R_2C-15[F
EEG T4-R_2B-12[T
EEG T6-R_2A-10[T
EEG F4-R_3C-16[F
EEG C4-R_3B-14[C
EEG P4-R_3A-11[P
EEG O2-R_4-13[O2
EEG A1-R_1-1[A1]
EEG F7-R_2C-7[F7
EEG T3-R_2B-4[T3
EEG T5-R_2A-2[T5
EEG F3-R_3C-8[F3
EEG C3-R_3B-6[C3
EEG P3-R_3A-3[P3
EEG O1-R_4-5[O1]
Sampling frequency:  16000.0
Downsampling by the factor  8
New sampling frequency:  2000.0
Reading done, 133.7 sec
Started creating bipolar signals

R1-2C created
R1-2B created
R1-2A created
R2C-2B created
R2B-2A created
R2A-2C created
R2C-3C created
R2B-3B created
R2A-3A created
R3C-3B created
R3B-3A created
R3A-3C created
R4-3C created
R4-3B created
R4-3A created
L1-2C created
L1-2B created
L1-2A created
L2C-2B created
L2B-2A created
L2A-2C created
L2C-3C created
L2B-3B created
L2A-3A created
L3C-3B created
L3B-3A created
L3A-3C created
L4-3C created
L4-3B created
L4-3A created


  0%|          | 0/30 [00:00<?, ?it/s]

In [7]:
p2.display_all_annotations()

----1Day----
------OFF------
    Onset  Duration             State
0    90.0     180.0            [Rest]
1   273.0      14.0    [LH, (Com), 1]
2   287.0       9.0  [LH, (NoCom), 1]
3   356.0      14.0    [RH, (Com), 1]
4   371.0      10.0  [RH, (NoCom), 1]
5   391.0      16.0    [LH, (Com), 2]
6   407.5      22.0  [LH, (NoCom), 2]
7   436.5      17.0    [RH, (Com), 2]
8   453.5      26.0  [RH, (NoCom), 2]
9   495.0      30.0      [LH, (Pass)]
10  535.0      30.0      [RH, (Pass)]
------ON------
     Onset  Duration             State
11  3768.0     180.0            [Rest]
12  4048.0      24.0    [RH, (Com), 1]
13  4106.0      18.0  [RH, (NoCom), 1]
14  4145.0      25.0    [RH, (Com), 2]
15  4203.0      26.0  [RH, (NoCom), 2]
16  4263.0      30.0   [RH, (Pass), 1]
17  4324.3      30.0   [RH, (Pass), 2]
18  4401.5      30.0    [LH, (Com), 1]
19  4467.5      26.0  [LH, (NoCom), 1]
20  4528.0      30.0    [LH, (Com), 2]
21  4591.0      30.0  [LH, (NoCom), 2]
22  4653.5      30.0   [LH, (Pas

In [8]:
# For this patient 1Day without movement-related conditions (not enough data for 180sec)
new_condition_name = "1Day OFF Rest 180sec"
conditions_to_merge = ["1Day OFF Rest"]
p2.merge_conditions(conditions_to_merge, new_condition_name, total_duration=180)

new_condition_name = "1Day ON Rest 180sec"
conditions_to_merge = ["1Day ON Rest"]
p2.merge_conditions(conditions_to_merge, new_condition_name, total_duration=180)


# new_condition_name = "1Day OFF RH Move 180sec"
# conditions_to_merge = ["1Day OFF " + s for s in ["RH (Com)", "RH (NoCom)", "RH (Hold)", "RH (Pass)"]]
# p2.merge_conditions(conditions_to_merge, new_condition_name, total_duration=180)


# new_condition_name = "1Day OFF LH Move 180sec"
# conditions_to_merge = ["1Day OFF " + s for s in ["LH (Com)", "LH (NoCom)", "LH (Hold)", "RHLH"]]
# p2.merge_conditions(conditions_to_merge, new_condition_name, total_duration=180)


# new_condition_name = "1Day ON RH Move 180sec"
# conditions_to_merge = ["1Day ON " + s for s in ["RH (Com)", "RH (NoCom)", "RH (Hold)", "RH (Pass)"]]
# p2.merge_conditions(conditions_to_merge, new_condition_name, total_duration=180)

# new_condition_name = "1Day ON LH Move 180sec"
# conditions_to_merge = ["1Day ON " + s for s in ["LH (Com)", "LH (NoCom)", "LH (Hold)", "LH (Pass)"]]
# p2.merge_conditions(conditions_to_merge, new_condition_name, total_duration=180)


## 5 DAY

new_condition_name = "5Day OFF Rest 180sec"
conditions_to_merge = ["5Day OFF Rest"]
p2.merge_conditions(conditions_to_merge, new_condition_name, total_duration=180)

new_condition_name = "5Day ON Rest 180sec"
conditions_to_merge = ["5Day ON Rest"]
p2.merge_conditions(conditions_to_merge, new_condition_name, total_duration=180)


new_condition_name = "5Day OFF RH Move 180sec"
conditions_to_merge = ["5Day OFF " + s for s in ["RH (Com)", "RH (NoCom)", "RH (Hold)", "RH (Pass)"]]
p2.merge_conditions(conditions_to_merge, new_condition_name, total_duration=180)


new_condition_name = "5Day OFF LH Move 180sec"
conditions_to_merge = ["5Day OFF " + s for s in ["LH (Com)", "LH (NoCom)", "LH (Hold)", "LH (Pass)"]]
p2.merge_conditions(conditions_to_merge, new_condition_name, total_duration=180)


new_condition_name = "5Day ON RH Move 180sec"
conditions_to_merge = ["5Day ON " + s for s in ["RH (Com)", "RH (NoCom)", "RH (Hold)", "RH (Pass)"]]
p2.merge_conditions(conditions_to_merge, new_condition_name, total_duration=180)

new_condition_name = "5Day ON LH Move 180sec"
conditions_to_merge = ["5Day ON " + s for s in ["LH (Com)", "LH (NoCom)", "LH (Hold)", "LH (Pass)"]]
p2.merge_conditions(conditions_to_merge, new_condition_name, total_duration=180)

Adding LFP to Patient2 object. 
Condition: 1Day OFF Rest 180sec 
Placement: R3C-3B
Updating condition
Adding LFP to Patient2 object. 
Condition: 1Day OFF Rest 180sec 
Placement: L4-3C
Updating condition
Adding LFP to Patient2 object. 
Condition: 1Day OFF Rest 180sec 
Placement: R2A-3A
Updating condition
Adding LFP to Patient2 object. 
Condition: 1Day OFF Rest 180sec 
Placement: L2A-2C
Updating condition
Adding LFP to Patient2 object. 
Condition: 1Day OFF Rest 180sec 
Placement: L4-3B
Updating condition
Adding LFP to Patient2 object. 
Condition: 1Day OFF Rest 180sec 
Placement: R2B-2A
Updating condition
Adding LFP to Patient2 object. 
Condition: 1Day OFF Rest 180sec 
Placement: R3A-3C
Updating condition
Adding LFP to Patient2 object. 
Condition: 1Day OFF Rest 180sec 
Placement: L2A-3A
Updating condition
Adding LFP to Patient2 object. 
Condition: 1Day OFF Rest 180sec 
Placement: L1-2B
Updating condition
Adding LFP to Patient2 object. 
Condition: 1Day OFF Rest 180sec 
Placement: R4-3C
Upd

In [10]:
for condition in p2.conditions:
    if "180sec" in condition:
        print(condition)

5Day ON RH Move 180sec
5Day OFF Rest 180sec
1Day OFF Rest 180sec
5Day ON LH Move 180sec
1Day ON Rest 180sec
5Day OFF LH Move 180sec
5Day ON Rest 180sec
5Day OFF RH Move 180sec


In [11]:
p2.save()

Saving Patient2 object to C:\Users\aleks\[[Python]]\LAB\Patient2\Patient2.pkl ...
Pickling Patient2 without ['pac']
Done, 2.187377691268921 sec
File size: 1852.8769359588623 MB
Returning filepath for saved file


'C:\\Users\\aleks\\[[Python]]\\LAB\\Patient2\\Patient2.pkl'